## 1.1 Bibliotecas

In [2]:
import pandas as pd
from pathlib import Path

## 2.2 Datos Titulados 2024 (Entrega 2)

In [3]:
def filtrar_y_agrupar_carreras(csv_file: str, min_titulados: int = 50, output_filename: str = '') -> pd.DataFrame:
    """
    Filtra y agrupa carreras de un DataFrame de titulados de educación superior.

    Args:
        csv_file (str): La ruta del archivo CSV a procesar.
        min_titulados (int): El número mínimo de titulados para que una carrera agrupada se mantenga en el resultado.

    Returns:
        pd.DataFrame: DataFrame filtrado y con la columna 'carrera_agrupada'.
    """

    try:
        df = pd.read_csv(csv_file, sep=';')
    except FileNotFoundError:
        print(f"Error: El archivo '{csv_file}' no fue encontrado.")
        return pd.DataFrame()
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        return pd.DataFrame()

    df_busqueda = df[['cat_periodo', 'nomb_carrera', 'nivel_carrera_1',
                      'nombre_titulo_obtenido', 'nomb_inst', 'tipo_inst_1',
                      'region_sede', 'area_conocimiento']].copy()

    # Filtrar por tipo de institución (Universidades)
    df_busqueda = df_busqueda[df_busqueda.tipo_inst_1 == 'Universidades']

    # Filtrar por nivel de carrera
    grados_objetivo = [
        'Licenciatura No Conducente a Título',
        'Profesional Con Licenciatura',
        'Profesional Sin Licenciatura'
    ]
    df_busqueda = df_busqueda[df_busqueda.nivel_carrera_1.isin(grados_objetivo)]

    # Filtrar por área de conocimiento (excluyendo Derecho, Salud, etc.)
    areas_objetivo = [
        'Tecnología',
        'Humanidades',
        'Educación',
        'Ciencias Básicas',
        'Agropecuaria',
        'Administración y Comercio'
    ]
    df_busqueda = df_busqueda[df_busqueda.area_conocimiento.isin(areas_objetivo)]

    # Aseguramos que 'nomb_carrera' esté en mayúsculas para la normalización
    df_busqueda['nomb_carrera'] = df_busqueda['nomb_carrera'].str.upper()

    # Filtrar por palabras clave de inclusión (INCLUIR)
    keywords_incluir = [
        'Periodismo', 'Letras', 'Literatura', 'Ingenieria', 'Estadistica',
        'Matematica', 'Fisica', 'Astronomia', 'Contador', 'Auditor', 'Lenguaje'
    ]
    # No es necesario upper() si ya lo hicimos antes, pero se mantiene la lógica
    regex_pattern_incluir = '|'.join(keywords_incluir).upper()

    mascara_incluir = df_busqueda.nomb_carrera.str.contains(regex_pattern_incluir, na=False)
    df_busqueda = df_busqueda[mascara_incluir].copy() # Usar copy() para evitar SettingWithCopyWarning

    # Filtrar por palabras clave de exclusión (EXCLUIR)
    keywords_excluir = [
        'BASICA', 'EDUCACION', 'TECNICO', "DEPORTE", "ACTIVIDAD", 'CREATIVA', 'INGLESAS'
    ]
    regex_pattern_excluir = '|'.join(keywords_excluir).upper()

    mascara_excluir = df_busqueda.nomb_carrera.str.contains(regex_pattern_excluir, na=False)
    df_busqueda = df_busqueda[~mascara_excluir].copy()

    # Crear la columna de 'carrera_agrupada'
    df_busqueda['carrera_agrupada'] = df_busqueda['nomb_carrera']

    # Agrupar carreras específicas

    # Ingeniería y Auditoría
    mascara_ingenieria = df_busqueda['nomb_carrera'].str.contains('INGENIERIA', na=False)
    mascara_auditoria = df_busqueda['nomb_carrera'].str.contains('AUDITOR', na=False)

    df_busqueda.loc[mascara_ingenieria, 'carrera_agrupada'] = 'INGENIERIA'
    df_busqueda.loc[mascara_auditoria, 'carrera_agrupada'] = 'AUDITOR'

    # Letras
    letras = [
        'LICENCIATURA EN LINGÜISTICA Y LITERATURA',
        'LETRAS HISPANICAS',
        'LICENCIATURA EN LITERATURA',
        'LICENCIATURA EN LENGUA Y LITERATURA', 'LICENCIATURA EN LETRAS MENCION LINGÜISTICA Y LITERATURA HISPANICA'
    ]
    df_busqueda.loc[df_busqueda['carrera_agrupada'].isin(letras), 'carrera_agrupada'] = 'LICENCIATURA EN LETRAS HISPANICAS'

    # Pedagogía en Matemáticas
    pedagogia_matematica = [
        'PEDAGOGIA EN MATEMATICAS Y FISICA', 'PEDAGOGIA EN MATEMATICAS Y/O LICENCIATURA EN MATEMATICAS',
        'PEDAGOGIA EN MATEMATICA', 'PEDAGOGIA EN MATEMATICAS E INFORMATICA EDUCATIVA',
        'PEDAGOGIA EN MATEMATICA Y COMPUTACION', 'PEDAGOGIA EN MATEMATICAS'
    ]
    df_busqueda.loc[df_busqueda['carrera_agrupada'].isin(pedagogia_matematica), 'carrera_agrupada'] = 'PEDAGOGIA EN MATEMATICAS'

    # Astronomía
    astronomia = [
        'LICENCIATURA EN ASTRONOMIA', 'ASTRONOMIA'
    ]
    df_busqueda.loc[df_busqueda['carrera_agrupada'].isin(astronomia), 'carrera_agrupada'] = 'ASTRONOMIA'

    # Filtrar por mínimo de titulados
    df_filtrado = df_busqueda.groupby('carrera_agrupada').filter(lambda x: len(x) >= min_titulados).copy()

    print(f"\n Conteo de Titulados por Carrera Agrupada (Mínimo {min_titulados} titulados):")

    try:
        df_filtrado.to_csv(output_filename, index=False, sep=';', encoding='utf-8')
        print(f"\n El DataFrame final ha sido guardado como: {output_filename}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

    return df_filtrado

In [4]:
dfs_titulados = []

for n in range(5):
    ruta = str(Path('datasets') / 'titulados' / f'Titulados_filtrado_202{n}.csv')
    dfs_titulados.append(pd.read_csv(ruta, sep=';'))

Hacemos unos ultimos ajustes

In [5]:
df_titulados_completo = pd.concat(dfs_titulados)
df_titulados_completo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169731 entries, 0 to 35563
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   cat_periodo             169731 non-null  int64 
 1   nomb_carrera            169731 non-null  object
 2   nivel_carrera_1         169731 non-null  object
 3   nombre_titulo_obtenido  168366 non-null  object
 4   nomb_inst               169731 non-null  object
 5   tipo_inst_1             169731 non-null  object
 6   region_sede             169731 non-null  object
 7   area_conocimiento       169731 non-null  object
 8   carrera_agrupada        169731 non-null  object
dtypes: int64(1), object(8)
memory usage: 12.9+ MB


Convertimos columnas a tipos correctos

In [6]:
df_titulados_completo.region_sede = df_titulados_completo.region_sede.astype('category')
df_titulados_completo.nivel_carrera_1 = df_titulados_completo.nivel_carrera_1.astype('category')
df_titulados_completo.area_conocimiento = df_titulados_completo.area_conocimiento.astype('category')
df_titulados_completo.carrera_agrupada = df_titulados_completo.carrera_agrupada.astype('category')

In [7]:
df_titulados_completo = df_titulados_completo.drop('tipo_inst_1', axis=1)
df_titulados_completo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169731 entries, 0 to 35563
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   cat_periodo             169731 non-null  int64   
 1   nomb_carrera            169731 non-null  object  
 2   nivel_carrera_1         169731 non-null  category
 3   nombre_titulo_obtenido  168366 non-null  object  
 4   nomb_inst               169731 non-null  object  
 5   region_sede             169731 non-null  category
 6   area_conocimiento       169731 non-null  category
 7   carrera_agrupada        169731 non-null  category
dtypes: category(4), int64(1), object(3)
memory usage: 7.1+ MB


In [11]:
ruta = str(Path('datasets') / 'limpios' / 'titulados.parquet')
df_titulados_completo.to_parquet(ruta)